# Week 2 Data Manipulation - Data Assembly, Missing Data and Tidy Data

Week 2 reading: **Pandas for Everyone** chapters 4 & 5 (pages 93 - 121)

**Outline**

* Chapter 4 - Data Assemply
    1. Intro to Tidy Data
    2. Adding Rows
    3. SQL Interlude -- Untderstanding Joins
    4. Adding Columns
    5. Merging Data
* Chapter 5 - Missing Data
    1. What is missing data and where does it come from?
    2. Dealing with missing data
    

## Overview

Machine learning models are fit from single tables but many times data will come to us as meltiple CSV files or even results of multiple database queries. The **tidy data** concept introduced in Chapter 4 is the end result of adding rows and columns, merging dataframes, and dealing with missing data. 

## 1. Chapter 4 - Data Assembly
### 1.1 Tidy Data

The idea of "Tidy Data" comes from RStudio developer Hadley Wickham in an article by the same name, published in 2014 by the Journal of Statistical Software. Wickham defined a "framework" set of 3 characteristics that all "tidy," or easily analyzable, data share (Tidy Data, Section 2.3): 

1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

Wickham makes the observation that his three rules correspond to Edgar F. Codd's **3rd Normal Form (3NF)** that forms the backbone of relational database design (Wickham, 2014). We'll discuss relevant parts of 3NF in Section 1.3 below, and look at Tidy Data in more detail as we discuss Exploratory Data Analysis.






### 1.2 Adding Rows

Section 4.3.1 discusses adding (concatenating) rows from three dataframes with identical column names, adding a series to a dataframe as well as converting the series to a dataframe before adding, and finally, adding dataframes while ignoring the indexes so numbering is continuous.

Some quick examples follow.

In [1]:
import pandas as pd
df1 = pd.read_csv("../pandas_for_everyone/data/concat_1.csv")
df2 = pd.read_csv("../pandas_for_everyone/data/concat_2.csv")
df3 = pd.read_csv("../pandas_for_everyone/data/concat_3.csv")

print(f'{df1}\n\n{df2}\n\n{df3}')

    A   B   C   D
0  a0  b0  c0  d0
1  a1  b1  c1  d1
2  a2  b2  c2  d2
3  a3  b3  c3  d3

    A   B   C   D
0  a4  b4  c4  d4
1  a5  b5  c5  d5
2  a6  b6  c6  d6
3  a7  b7  c7  d7

     A    B    C    D
0   a8   b8   c8   d8
1   a9   b9   c9   d9
2  a10  b10  c10  d10
3  a11  b11  c11  d11


In [2]:
print(pd.concat([df1, df2, df3]))

     A    B    C    D
0   a0   b0   c0   d0
1   a1   b1   c1   d1
2   a2   b2   c2   d2
3   a3   b3   c3   d3
0   a4   b4   c4   d4
1   a5   b5   c5   d5
2   a6   b6   c6   d6
3   a7   b7   c7   d7
0   a8   b8   c8   d8
1   a9   b9   c9   d9
2  a10  b10  c10  d10
3  a11  b11  c11  d11


In [3]:
series_row = pd.Series(['n1', 'n2', 'h3', 'n4'])

print(pd.concat([df1, series_row]))

     A    B    C    D    0
0   a0   b0   c0   d0  NaN
1   a1   b1   c1   d1  NaN
2   a2   b2   c2   d2  NaN
3   a3   b3   c3   d3  NaN
0  NaN  NaN  NaN  NaN   n1
1  NaN  NaN  NaN  NaN   n2
2  NaN  NaN  NaN  NaN   h3
3  NaN  NaN  NaN  NaN   n4


In [4]:
new_df = pd.DataFrame([['n1', 'n2', 'h3', 'n4']], columns = ['A','B','C','D'])
print(new_df)

    A   B   C   D
0  n1  n2  h3  n4


Besides the concat() function (for adding multiple things), you can also use append() to add just one object: 

In [5]:
print(df1.append(df2))

    A   B   C   D
0  a0  b0  c0  d0
1  a1  b1  c1  d1
2  a2  b2  c2  d2
3  a3  b3  c3  d3
0  a4  b4  c4  d4
1  a5  b5  c5  d5
2  a6  b6  c6  d6
3  a7  b7  c7  d7


Using `ignore_index=True` will allow the new dataframe to index properly:

In [6]:
print(df1.append(df2, ignore_index=True))

    A   B   C   D
0  a0  b0  c0  d0
1  a1  b1  c1  d1
2  a2  b2  c2  d2
3  a3  b3  c3  d3
4  a4  b4  c4  d4
5  a5  b5  c5  d5
6  a6  b6  c6  d6
7  a7  b7  c7  d7


### 1.3 SQL Interlude -- Understanding Joins

To understand joins in SQL, we should probably have a rudimentary understanding of *why* we have to join data.

Consider an employee table in a fictitious company:

name | department 
-----|-----------
Tom| HR
Mary | Development
John | Marketing
Tim | H.R.

Notice that both Tom and Tim work in HR. **What happens if we want to change 'HR' to 'Human Resources?'** Of course it is easy to see we would change the department on those two rows, But what if we had 1,000 employees in our table? How would be find them all? Also notice how 'HR' is actually entered two different ways. Which one is correct? What happens if both Tim and Tom get fired and we delete those two rows? We lose the fact that an HR department even exists!

To combat these problems, we **normalize** the table above into two tables, employees and departments.

**employees**
emp_id | emp_name | dept_id
--|-----|-----------
1 | Tom | 1
2 | Mary | 2
3 | John | 3
4 | Tim | 1
5 | Jenny | 

NOTE: Jenny is a new hire not assigned a department yet.<br>
**departments**

dept_id | dept_name
--------|----------
1 | HR
2 | Development
3 | Marketing

By normalizing the employee table, we've eliminated the source of many errors as well as a lot of repeated data, **but** now we can't just look at one table to get all the employees -- department 3 won't mean anything to anyone other than than a database administrator. How do we solve this problem?



#### Enter Joins

As you can see in figure, there are many types of join:<br>
<img align="right" style="padding-right:10px;" src="figures/500px-SQL_Joins_labels.png"><br>

In the world of database queries, the top 3 are by far the most common:
* Inner join
    * Return only the stuff that matches in both tables
* Left join
    * Return all the stuff in the left table and stuff from the right that matches a key (like an id)
* Right join
    * Return all the stuff in the right table and stuff from the left that matches a key




**Left Join**<br>
`select employees.name, d.dept_name from employees left join departments d on employees.dept_id = d.dept_id;`

Don't worry about the SQL syntax. The result of the left join is:

name | dept_name
-----|-----------
Tom | HR
Mary | Development
John | Marketing
Tim | HR
Jenny | null
    
notice that left join gives us everything from the left table (employees) even if there is no match -- Jenny has invisible null for dept_name.
    
<hr>

**Inner Join**<br>
`select employees.name, d.dept_name from employees inner join departments d on employees.dept_id = d.dept_id;`

name | dept_name
-----|-----------
Tom | HR
Mary | Development
John | Marketing
Tim | HR
    
Inner join gives us what we expect, only the employees that have a matching dept_id in the departments table.
    
<hr>

**Right Join**<br>
Right join acts the same way as left join, with the effects reversed. To see it, let's add *sales* and *engineering* departments.

`select employees.name, d.dept_name from employees right join departments d on employees.dept_id = d.dept_id;`

name | dept_name
-----|-----------
Tom | HR
Mary | Development
John | Marketing
Tim | HR
null | Sales
null |engineering


### 1.4 Adding Columns

For the most part, adding columns with the concat() function works the same as rows, except we have to specify axis=1.

One thing the book doesnot really mention at this stage is adding columns simply by naming them. Like so:

In [9]:
print(f'{df1}\n\n{df2}')

    A   B   C   D
0  a0  b0  c0  d0
1  a1  b1  c1  d1
2  a2  b2  c2  d2
3  a3  b3  c3  d3

    A   B   C   D
0  a4  b4  c4  d4
1  a5  b5  c5  d5
2  a6  b6  c6  d6
3  a7  b7  c7  d7


In [14]:
df1['E'] = df2['D']
df1

,A,B,C,D,E
0,a0,b0,c0,d0,d4
1,a1,b1,c1,d1,d5
2,a2,b2,c2,d2,d6
3,a3,b3,c3,d3,d7


In most cases, that will be the easiest way to add a column.

Pandas does support the "join" method of concatenating or merging dataframes.

In [34]:
# **employees**
# emp_id | emp_name | dept_id
# --|-----|-----------
# 1 | Tom | 1
# 2 | Mary | 2
# 3 | John | 3
# 4 | Tim | 1
# 5 | Jenny | 

# NOTE: Jenny is a new hire not assigned a department yet.<br>
# **departments**

# dept_id | dept_name
# --------|----------
# 1 | HR
# 2 | Development
# 3 | Marketing
import numpy as np

# Have to put a 0 in for Jenny, then adjust it.
emp_df = pd.DataFrame({'emp_id':[1,2,3,4,5], 'emp_name':['Tom', 'Mary','John', 'Tim', 'Jenny'], 'dept_id':[1,2, 3, 1, 0] })
dept_df = pd.DataFrame({'dept_id':[1,2,3], 'dept_name':['HR','Development', 'Marketing']})

# Workaround to get a null into Jenny's dept_id
# emp_df.loc[emp_df['emp_name'] == 'Jenny', 'dept_id'] = np.nan

print(f'{emp_df}\n\n{dept_df}')

   emp_id emp_name  dept_id
0       1      Tom        1
1       2     Mary        2
2       3     John        3
3       4      Tim        1
4       5    Jenny        0

   dept_id    dept_name
0        1           HR
1        2  Development
2        3    Marketing


In [40]:
print(pd.concat([emp_df, dept_df], join='outer', axis=1))

   emp_id emp_name  dept_id  dept_id    dept_name
0       1      Tom        1      1.0           HR
1       2     Mary        2      2.0  Development
2       3     John        3      3.0    Marketing
3       4      Tim        1      NaN          NaN
4       5    Jenny        0      NaN          NaN


In [39]:
print(pd.concat([emp_df, dept_df],  join='inner', axis=1))

   emp_id emp_name  dept_id  dept_id    dept_name
0       1      Tom        1        1           HR
1       2     Mary        2        2  Development
2       3     John        3        3    Marketing


However, concatenating is simply smashing two dataframes together based on index. Let's look at more of a true join with the merge() function.

### 1.5 Merging Data

The book does a good job of describing database concepts of "one-to-one", "many-to-one", and "many-to-many." Our employee table is a one-to-one relationship -- one employee works in one department. 

The book describes parameters that can be used if column names are different, but in this case the developer was plannning ahead and made sure they match :).

In [43]:
emp_df.merge(dept_df, on="dept_id")

,emp_id,emp_name,dept_id,dept_name
0,1,Tom,1,HR
1,4,Tim,1,HR
2,2,Mary,2,Development
3,3,John,3,Marketing


Notice how Jenny, whose department ID doesn't match in the departments table, is omitted. **Inner join** is the default. https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/ demonstrates concatenation and merging very well.

## 2. Chapter 5 Missing Data
### 2.1 What is missing data and where does it come from?

**What**<br>
Conceptually, missing values in the data are exactly that - missing. Nothing there. Not "0" and not " " (blank space). Non-existance.

Programming languages generally have the concept of a "null" value, although a different word may be used. For example, pure Python uses the keyword *"None"* to express the concept. 

Much of Pandas is based on the NumPy high-performance mathematical library. As such, Pandas and NumPy both use **NaN** -- "Not a Number" to express missing or null values.

**Where**<br>
NaN or null values can creep into a data set in a variety of ways:

* A value was not entered or entered incorrectly on a form.
* An observation from a sensor was corrupt or missing.
* Transcription error or typo.
* Etc. 

**But wait, there's more**<br>
A much more insidious and hard to detect type of "null" comes in the form of a substitute value. Perhaps the data was stored in a database that didn't allow nulls (common) or due to a data-entry convention, or some other reason, missing data is not allowed to be null.

A prime example of this is the **"Heart Disease Data Set"** from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Heart+Disease). This data suffers from a number of problems, including the fact that it is delimited by spaces and one observation takes multiple rows (10). Pandas can't read it in its native form, and this data set will be the subject of scrutiny in MSDS 621. For now, let's just load the file and look at one observation (one row).

In [45]:
hrt_data = []
with open ('data/hungarian.data', 'r') as infile:
    for line in infile:
        hrt_data.append(line)

print(hrt_data[:10])

['1254 0 40 1 1 0 0\n', '-9 2 140 0 289 -9 -9 -9\n', '0 -9 -9 0 12 16 84 0\n', '0 0 0 0 150 18 -9 7\n', '172 86 200 110 140 86 0 0\n', '0 -9 26 20 -9 -9 -9 -9\n', '-9 -9 -9 -9 -9 -9 -9 12\n', '20 84 0 -9 -9 -9 -9 -9\n', '-9 -9 -9 -9 -9 1 1 1\n', '1 1 -9. -9. name\n']


Data like the file above is a good example why you need to learn about your data. These -9s are the only negative numbers in the data set, but there are a lot of them. If they were allowed to stay -9s, they would drastically affect any analysis or machine learning algorithm.

Pages 109 - 116 of **"Pandas for Everyone"** does a good job of showing NaN values and the properties they (don't) possess as well as various ways they can sneak into perfectly respectable data sets through joins, user input, re-indexing, etc.

### 2.2 Finding and Dealing with missing data

What is that saying? *"The first step is admitting you have a problem?"* **"Pandas for Everyone"** discusses one method of finding nulls based on column counts and data set shape. We will present a slightly different method so you can choose which method you prefes.

First, we will load in the data file:

In [49]:
ebola = pd.read_csv("../pandas_for_everyone/data/country_timeseries.csv")
ebola.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


Simply looking at the first few rows clearly shows we have NaNs. 

Next, we will use the Pandas info() function to tell us about the columns:

In [48]:
ebola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 18 columns):
Date                   122 non-null object
Day                    122 non-null int64
Cases_Guinea           93 non-null float64
Cases_Liberia          83 non-null float64
Cases_SierraLeone      87 non-null float64
Cases_Nigeria          38 non-null float64
Cases_Senegal          25 non-null float64
Cases_UnitedStates     18 non-null float64
Cases_Spain            16 non-null float64
Cases_Mali             12 non-null float64
Deaths_Guinea          92 non-null float64
Deaths_Liberia         81 non-null float64
Deaths_SierraLeone     87 non-null float64
Deaths_Nigeria         38 non-null float64
Deaths_Senegal         22 non-null float64
Deaths_UnitedStates    18 non-null float64
Deaths_Spain           16 non-null float64
Deaths_Mali            12 non-null float64
dtypes: float64(16), int64(1), object(1)
memory usage: 17.2+ KB


Notice the info() function tells us how many how many rows we have ( `RangeIndex: 122 entries` ), how many non-null values are in each column, and what type of data the column contains.

The book proposes several standard ways of handling the missing data:
* Recode/replace (recoding as 0 is used as an example)
* Fill forward using the last known good value
* Fill backward using the next good value
* Interpolation (by equally spacing values)
* Dropping rows with missing values

Since much of the time we are doing our data wrangling as a precursor to building a machine learning model, we will take a quick look at scikit-learn's SimpleImputer that can be used for both numeric and categorical data, alone or as part of a preprocessing pipeline.

SimpleImputer can recognize both NaNs and substitute values such as our -9s above and replace them with *a constant value* or:

* mean for the column
* median for the column
* most frequent value for the column

Let's demonstrate on the ebola dataframe we loaded earlier:

In [50]:
from sklearn.impute import SimpleImputer

Scikit-learn is built to use NumPy 2D matrixes of numbers, so we will make our data set, X, out of the numeric values and check a slice:

In [64]:
X = ebola.iloc[:,2:].values
X[:5,:]

array([[ 2776.,    nan, 10030.,    nan,    nan,    nan,    nan,    nan,
         1786.,    nan,  2977.,    nan,    nan,    nan,    nan,    nan],
       [ 2775.,    nan,  9780.,    nan,    nan,    nan,    nan,    nan,
         1781.,    nan,  2943.,    nan,    nan,    nan,    nan,    nan],
       [ 2769.,  8166.,  9722.,    nan,    nan,    nan,    nan,    nan,
         1767.,  3496.,  2915.,    nan,    nan,    nan,    nan,    nan],
       [   nan,  8157.,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,  3496.,    nan,    nan,    nan,    nan,    nan,    nan],
       [ 2730.,  8115.,  9633.,    nan,    nan,    nan,    nan,    nan,
         1739.,  3471.,  2827.,    nan,    nan,    nan,    nan,    nan]])

Next, we will create our imputer, fit it to the data, then transform the data.

In [66]:
np.set_printoptions(suppress=True) # Suppresses scientific notation for small numbers
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)  
X_new = imputer.transform(X)
print(X_new[:5,:])

[[ 2776.          2335.3373494  10030.            16.73684211
      1.08           3.27777778     1.             3.5
   1786.          1101.20987654  2977.             6.13157895
      0.             0.83333333     0.1875         3.16666667]
 [ 2775.          2335.3373494   9780.            16.73684211
      1.08           3.27777778     1.             3.5
   1781.          1101.20987654  2943.             6.13157895
      0.             0.83333333     0.1875         3.16666667]
 [ 2769.          8166.          9722.            16.73684211
      1.08           3.27777778     1.             3.5
   1767.          3496.          2915.             6.13157895
      0.             0.83333333     0.1875         3.16666667]
 [  911.06451613  8157.          2427.36781609    16.73684211
      1.08           3.27777778     1.             3.5
    563.23913043  3496.           693.70114943     6.13157895
      0.             0.83333333     0.1875         3.16666667]
 [ 2730.          8115.         

As you can see, there are no more NaNs in sight.

Transformations such as scaling, normalization, and one-hot encoding (of categorical columns) are very common operations leading up to fitting a machine learning model. So common, in fact, that scikit-learn created a **pipeline** module to help bundle the transformations as a series of steps.

For more information, please see the scikit-learn documentation:

https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

https://scikit-learn.org/stable/data_transforms.html

*References:*

Wickham, H. (2014). Tidy data. Journal of Statistical Software, 59(10), 1-23.